# Sensitivity Analysis

Made by Laura van Geene, Spring 2024, for MSc thesis research.

File must be run in combination with 'full_model_greenlight.py'

Code requires data files to run:
- Figaro IO-table (download from Figaro website)
- Figaro CO2 extensions (download from Figaro website)
- Figaro sector and region labels (region_labels.txt and sector_labels.txt)
- List of elasticities (for this research: Elasticities Thesis.xlsx)
- List of expenditures to include in analysis from CBS Dataset (main_expenditures.xlsx)
- Installation of Python library cbsodata (can be installed via pip)
- Concordance table for CBS and Figaro data (Concordance Table COICOP-Figaro.csv)

Put datafiles in folder called 'data'

In [ ]:
import numpy as np
import pandas as pd
import cbsodata #for CBS data import
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib
import seaborn as sns

In [ ]:
#import analysis functions
from full_model_greenlight import import_figaro, CBS_Data, do_input_output_energy, do_input_output_NL, concordance
from full_model_greenlight import do_analysis, do_analysis_energy, do_consumption_analysis, do_consumption_analysis_energy

In [ ]:
#set up file pats
filepath="data/Figaro/"
filepath2="data/"

In [ ]:
#import IO data and calculate MRIO variables
A, A_old, L, A_trans, L_trans, inv_diag_x_, x, Y, v0, fco2, multi_index, start_NL, end_NL, v_df, sector_labels = import_figaro(filepath)

In [ ]:
#import the consumption table from CBS
consumption_table=CBS_Data(filepath2)

In [ ]:
#Import DNB Elasticities
Bun_elasticities=pd.read_excel(f'{filepath2}Elasticities Thesis.xlsx', sheet_name='BUN', usecols=[2], index_col=None)
Bun_elasticities=Bun_elasticities.squeeze() #turns into array
Bun_elas_list=Bun_elasticities.tolist() #turns into list

### Run conventional model

In [ ]:
sectors= sector_labels
the_prices=[0.05, 0.15, 0.25]
sigma_list=Bun_elas_list

In [ ]:
results_CPI_NL={} #CPI outcomes
results_LPM_NL={} #sectoral price changes
results_A_star_sub_NL={}
results_A_star_tech_NL={}
consumption_NL={}

In [ ]:
for price in the_prices:
    #calculate CPI
    result_NL, result_tax_NL, A_star_sub, A_star_tech = do_analysis_energy(price, sectors, sigma_list, filepath, filepath2, consumption_table, start_NL, end_NL, None, None)
    results_CPI_NL[f'{price}']=result_NL
    results_LPM_NL[f'{price}']= result_tax_NL
    results_A_star_sub_NL[f'{price}']=A_star_sub
    results_A_star_tech_NL[f'{price}']=A_star_tech
    
    #consumption analysis
    collapsed_LPM_NL=do_consumption_analysis(filepath, filepath2, price, sectors, sigma_list, start_NL, end_NL, None, None)
    consumption_NL[f'{price}']=collapsed_LPM_NL

In [ ]:
#Consumer Price Index
results_CPI_50_NL=results_CPI_NL['0.05']
results_CPI_150_NL=results_CPI_NL['0.15']
results_CPI_250_NL=results_CPI_NL['0.25']

#Sectoral price changes
results_50_LPM_NL=results_LPM_NL['0.05']
results_150_LPM_NL=results_LPM_NL['0.15']
results_250_LPM_NL=results_LPM_NL['0.25']

#collect consumption analysis results
consumption_50=consumption_NL['0.05']
consumption_50_NL=pd.Series(consumption_50.squeeze())
consumption_150=consumption_NL['0.15']
consumption_150_NL=pd.Series(consumption_150.squeeze())
consumption_250=consumption_NL['0.25']
consumption_250_NL=pd.Series(consumption_250.squeeze())

### Change elasticity for electricity sector

In [ ]:
results_CPI_150={} #CPI outcomes
results_LPM_150={} #sectoral price changes
results_A_star_sub_150={}
results_A_star_tech_150={}

In [ ]:
sectors = sector_labels
sigma_list = Bun_elas_list
the_prices = [0.15, 0.15, 0.15, 0.15]
the_weird_sectors = ["Electricity, gas, steam and air conditioning supply", "Electricity, gas, steam and air conditioning supply", "Electricity, gas, steam and air conditioning supply","Electricity, gas, steam and air conditioning supply"]
the_weird_sigmas = [0.3, 0.5, 0.7, 1.0]

for price, weird_sector, weird_sigma in zip(the_prices, the_weird_sectors, the_weird_sigmas):
    #print(price)
    #print(weird_sector)
    #print(weird_sigma)
    result_energy, result_tax_energy, A_star_sub, A_star_tech = do_analysis_energy(price, sectors, sigma_list, filepath, filepath2, consumption_table, start_NL, end_NL, weird_sector, weird_sigma)
    results_CPI_150[f'{weird_sigma}']=result_energy
    results_LPM_150[f'{weird_sigma}']=result_tax_energy

        

In [ ]:
#isolate CPIs for different elasticities
result_CPI_3_150=results_CPI_150['0.3'].iloc[:, -2:]
result_CPI_5_150=results_CPI_150['0.5'].iloc[:, -2:]
result_CPI_7_150=results_CPI_150['0.7'].iloc[:, -2:]
result_CPI_one_150=results_CPI_150['1.0'].iloc[:, -2:]

#isolate top 5 contributors
results_LPM_3_150=results_LPM_150["0.3"].loc["the Netherlands", :].nlargest(5, 'no substitution')
results_LPM_5_150=results_LPM_150['0.5'].loc["the Netherlands", :].nlargest(5, 'no substitution')
results_LPM_7_150=results_LPM_150['0.7'].loc["the Netherlands", :].nlargest(5, 'no substitution')
results_LPM_one_150=results_LPM_150['1.0'].loc["the Netherlands", :].nlargest(5, 'no substitution')

In [ ]:
#compile all results into big table
sensitivity_results_CPI=pd.concat([result_CPI_3_150, result_CPI_5_150, result_CPI_7_150, result_CPI_one_150], ignore_index=True)
sensitivity_results_CPI=pd.DataFrame(sensitivity_results_CPI.values, columns=["65 to 75", "75 and older"])

In [ ]:
#create appropriate index
elas=[0.3, 0.5, 0.7, 1.0]
substitution_cat=['without substitution', 'with capital/labour subst.']
big_multi_index=pd.MultiIndex.from_product([elas, substitution_cat], names=['Elasticity','Substitution Type'])

#make table
sensitivity_results_CPI=sensitivity_results_CPI.set_index(big_multi_index)
sensitivity_results_CPI.round(2)

In [ ]:
#make big table for sectoral price changes
sensitivity_results_LPM=pd.concat([results_LPM_3_150, results_LPM_5_150, results_LPM_7_150, results_LPM_one_150], ignore_index=True)

#make index
elas=[0.3, 0.5, 0.7, 1.0]
sect=['Electricity, gas, steam and air conditioning supply', 'Other professional, scientific and veterinary activities', 'Security, service and landscape, office support activities', 'Manufacture of basic metals', 'Activities of membership organisations']
big_multi_index=pd.MultiIndex.from_product([elas, sect], names=['Elasticity', 'Sector'])

#make overview table for results
sensitivity_results_LPM=pd.DataFrame(sensitivity_results_LPM.values, big_multi_index, columns=substitution_cat)
sensitivity_results_LPM=(sensitivity_results_LPM-1)*100
sensitivity_results_LPM.drop(["without substitution"], axis=1).round(2)